In [2]:
from config import w_key
import requests
import json
import pandas as pd

In [3]:
path = "Resources/us_station_id.csv"
coord_csv = pd.read_csv(path)
coord_csv

Station ID        Lat         Lng
0          25704  51.883583 -176.642481
1          26610  62.500000 -164.420000
2          45701  52.830000  173.320000
3          25502  56.899487 -154.247847
4          26515  66.551833 -152.622167
...          ...        ...         ...
3430       94053  42.064528 -104.152694
3431       24082  42.000000 -104.166700
3432       24083  42.066700 -104.966700
3433       24062  43.962889 -107.950528
3434       94173  44.550000 -110.416700

[3435 rows x 3 columns]

In [4]:
path = "Resources/coops-activecurrentstations.csv"
curr_csv = pd.read_csv(path)
curr_csv

Station ID                  Station Name   Latitude   Longitude  \
0      ca0101      Cape Cod Canal  West End  41.737499  -70.624901   
1      cb0102             Cape Henry LB 2CH  36.959400  -76.012800   
2      cb0201       York Spit LBB 22 (ATON)  37.140000  -76.138300   
3      cb0301           Thimble Shoal LB 18  37.011080  -76.249030   
4      cb0402    Naval Station Norfolk LB 7  36.962950  -76.333880   
..        ...                           ...        ...         ...   
60     sn0701                   Port Arthur  29.866700  -93.931110   
61     t01010   Sunshine Skyway Bridge ADCP  27.620650  -82.655250   
62     t02010                Old Port Tampa  27.862870  -82.553730   
63     t03010                  Port Manatee  27.663950  -82.598770   
64     t04010           Egmont Channel LB 9  27.604240  -82.867420   

                        Project  
0                Cape Cod PORTS  
1    Chesapeake Bay South PORTS  
2    Chesapeake Bay South PORTS  
3    Chesapeake Bay South PORTS  
4    Chesapeake Bay South PORTS  
..                          ...  
60          Sabine Neches PORTS  
61              Tampa Bay PORTS  
62              Tampa Bay PORTS  
63              Tampa Bay PORTS  
64              Tampa Bay PORTS  

[65 rows x 5 columns]

In [10]:
#TO TEST A DIFFERENT TYPE OF CURRENT API CALL.... Ignore this one

url = f"https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9414290/nearby.json?radius=3"
#url = f"https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/lat/51.883583/lng/-176.642481/nearby.json?radius=3"
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))





{
    "count": null,
    "self": "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/45701/nearby.json",
    "stations": [],
    "units": null
}


In [11]:
#TO TEST STATION IDS
url = f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?date=today&station=ca0101&product=currents_predictions&units=metric&time_zone=gmt&application=ports_screen&format=json"
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))
#print(response["current_predictions"]["cp"][0]["Velocity_Major"])






{
    "current_predictions": {
        "cp": [
            {
                "Bin": "20",
                "Depth": "4",
                "Time": "2020-12-02 00:00",
                "Velocity_Major": 164.1,
                "meanEbbDir": 217,
                "meanFloodDir": 41
            },
            {
                "Bin": "20",
                "Depth": "4",
                "Time": "2020-12-02 00:06",
                "Velocity_Major": 164.3,
                "meanEbbDir": 217,
                "meanFloodDir": 41
            },
            {
                "Bin": "20",
                "Depth": "4",
                "Time": "2020-12-02 00:12",
                "Velocity_Major": 164.4,
                "meanEbbDir": 217,
                "meanFloodDir": 41
            },
            {
                "Bin": "20",
                "Depth": "4",
                "Time": "2020-12-02 00:18",
                "Velocity_Major": 164.3,
                "meanEbbDir": 217,
                "meanFloodDir":

In [16]:

#Open weather query
units = "standard"
#lats = coord_csv["Lng"]
#lons = coord_csv["Lat"]

lats = ["51.883583"]
lons = ["-176.642481"]

#Outputs    
temps = []


for (lat, lon) in zip(lats, lons):
    w_url= f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units={units}&appid={w_key}"
    w_query= requests.get(w_url).json()
    temp_get = w_query['main']['temp']
    temps.append(temp_get)
    

    
    



In [17]:
#Tide query
date = "today"
product = "currents_predictions"


t_stations = curr_csv["Station ID"].to_list()
datum = "MTL"
unit = "metric"
time = "lst"
format_j = "json"
app = "app"

#Outputs
velo = []
ebb = []
flood = []
s_result = []


for t_station in t_stations:
    try:
        t_url = f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?date={date}&station={t_station}&product={product}&units={unit}&time_zone={time}&application={app}&format={format_j}"
        #t_url = f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?date={date}&station={station}&product={product}&datum={datum}&units={unit}&time_zone={time}&application={app}&format={format_j}"
        t_resp = requests.get(t_url).json()
        velocity = t_resp["current_predictions"]["cp"][0]["Velocity_Major"] #In cm/second
        tide_ebb = t_resp["current_predictions"]["cp"][0]["meanEbbDir"] #degrees azimuth (360)
        tide_flood = t_resp["current_predictions"]["cp"][0]["meanFloodDir"] #degrees azimuth
        velo.append(velocity)
        ebb.append(tide_ebb)
        flood.append(tide_flood)
        s_result.append(t_station)
    except KeyError:
        print(t_station)
    
    
    

    
    

cc0201
g10010
gl0101
gl0201
lc0101
lm0101
lm0201
mb0302
mc0101
mi0101
mi0201
sn0101
sn0401
sn0601
t04010


In [16]:
velo

AttributeError: 'list' object has no attribute 'value_counts'

In [14]:
s_result

['ca0101',
 'cb0102',
 'cb0201',
 'cb0301',
 'cb0402',
 'cb0601',
 'cb0701',
 'cb0801',
 'cb1001',
 'cb1101',
 'cb1201',
 'cb1301',
 'cc0301',
 'cc0401',
 'cp0101',
 'db0301',
 'db0502',
 'g06010',
 'g08010',
 'g09010',
 'hb0401',
 'jx0302',
 'jx0401',
 'jx0501',
 'jx0601',
 'jx0701',
 'kb0101',
 'kb0201',
 'kb0301',
 'kb0401',
 'lc0201',
 'lc0301',
 'mg0101',
 'mg0201',
 'n03020',
 'n06010',
 'n07010',
 'nb0301',
 'nl0101',
 's06010',
 's08010',
 's09010',
 's10010',
 'sn0201',
 'sn0301',
 'sn0501',
 'sn0701',
 't01010',
 't02010',
 't03010']